# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Week 3 Assessment

First, it is necessary to import all the required dependencies and libraries

In [1]:
#import
import pandas as pd
from bs4 import BeautifulSoup
import requests
import collections

We will use BeautifulSoup to retrieve the information from the specified Wikipedia page

In [2]:
#Alternatively, if we want to feed the webpage directly we can do the following:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
newsoup = BeautifulSoup(source, 'lxml') #create a new beautiful soup object
#print(newsoup.prettify())

Because the postal codes and neighbourhoods are in a table, we can just work with the tbody element from the website and then find all the table rows (trs)

In [3]:
tbl = newsoup.find('tbody') #find the table body element where all the information is embedded
trs= tbl.find_all('tr') #find all the table rows containing the information
#print(trs[0])

#from the first table row trs[0], obtain the table headings and save them in an array of strings
titles=trs[0].find_all('th')
title1=titles[0].text
title2=titles[1].text
title3=titles[2].text
title3=title3.split('\n')[0]
titles=[title1,title2,title3] #these are the titles for the dataframe colums
print(titles)

['Postcode', 'Borough', 'Neighbourhood']


We have to reorganize the information from the table so that it can be used to create a dataframe, for this we have created three empty lists: postcodes, boroughs and neighbourhoods

In [4]:

info = []
postcodes=[]
boroughs=[]
neighbourhoods=[]

for x in range(1,len(trs)):  #this would be the beginning of the code needed to iterate over the rest of the rows

#trying with the first row containing valuable informtaion:
    tds=trs[x].find_all('td') #from the row, find all the tds
    cell1=tds[0].text
    cell2=tds[1].text
    cell3=tds[2].text.split('\n')[0]
    if cell2 != 'Not assigned':
        if cell3 == 'Not assigned':
            cell3=cell2
        row1=[cell1,cell2,cell3]
        postcodes.append(cell1) #append the the first cell of the row (Postcode) to the postcodes list
        boroughs.append(cell2) #append the second cell of the row (Borough) to the boroughs list
        neighbourhoods.append(cell3) #append the third cell of the row (Neighbourhood) to the neighbourhoods list
        
        info.append(row1) #create a list with lists containing the rows of the Postcodes' table and append information
        
#print(info) #in case someone wants to look at the information collected from the website

Then we can create a dictionary using the previously mentioned lists (arrays), and pass this as an argument to create a pandas dataframe

In [5]:
columnas={'pcode':postcodes,'boroughs':boroughs,'neighbourhoods':neighbourhoods}
df=pd.DataFrame(columnas)
len(df.pcode.unique())

103

Because there are some neighbourhoods that have the same postcode, we use the groupby method and display the first five entries of the updated dataframe

In [6]:
df = df.groupby(['pcode','boroughs'])['neighbourhoods'].apply(', '.join).reset_index()
df.head()
#df.loc[df['pcode'] == 'M5A']

,pcode,boroughs,neighbourhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, we display the number of rows in the dataframe

In [8]:
df.shape

(103, 3)